# 05 Error metrics

The toolkit allows to evaluate standar error metrics automaticaly, as well as only obtaint the raw error values and compute error metrics not included in the toolkit.

This notebook will continue on the Air quality dataset, 

## Usage notes
This toolkit works in the following order:- Set up the input data
- Create model (SM, MOSM, CSM, SM-LMC, CG)
- Make an estimation of the model parameters using BNSE+SM
- Train the model parameters
- Set the prediction range
- Do a prediction For example:

#### Imports and plot format

In [ ]:
# execute if not installed
import sys
import os
sys.path.insert(0, '../')

import mogptk

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

# Air Quality MOGP


The dataset hourly averaged responses from an array of 5 metal oxide chemical sensors embedded in an Air Quality Chemical Multisensor Device.

The device was located on the field in a significantly polluted area, at road level,within an Italian city. Data were recorded from March 2004 to February 2005 (one year)representing the longest freely available recordings of on field deployed air quality chemical sensor devices responses.

We will only use 5 columns CO(GT), NMHC(GT), C6H6(GT), NOx(GT), NO2(GT).

The same as the previous tutorial [04-Model Training](https://github.com/GAMES-UChile/MultiOutputGP-Toolkit/blob/master/examples/04_Model_Training.ipynb) We load, the dataset, replace the sensor failures (-200) with nans and convert the date columns to hours.

In [ ]:
# load
df = pd.read_csv('data/AirQualityUCI.csv', delimiter=';')

cols = ['CO(GT)', 'NMHC(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)']

# replace missing values with nan
df.replace(-200.0, np.nan, inplace=True)

# First 2 columns are date and time, we convert it to a single column with datetime format
df['Date'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H.%M.%S')

# define a initial date to compare all other to
ini_date = pd.Timestamp('2004-03-10 00:00:00.0')

# get elapsed hours
df['Time'] = (df['Date'] - ini_date) / pd.Timedelta(hours=1)

# use only the first 8 days of data
df2 = df[df['Date'] < pd.Timestamp('2004-03-19 00:00:00.0')]

dataset = mogptk.LoadDataFrame(df2,
                    x_col='Time',
                    y_col=cols)

we will also use the transformation defined in the data prepararion and transformation tutorial [02-Data Preparation](https://github.com/GAMES-UChile/MultiOutputGP-Toolkit/blob/master/examples/02_Data_Preparation.ipynb) where each channel is normalized so it has mean 0 and variance 1.

In [ ]:
class Transform_Whiten:
    """
    Transform the data so it has mean 0 and variance 1
    """
    def __init__(self):
        pass
    
    def _data(self, data):
        # take only the non-removed observations
        self.mean = data.Y[data.mask].mean()
        self.std = data.Y[data.mask].std()
        
    def _forward(self, x, y):
        return (y - self.mean) / self.std
    
    def _backward(self, x, y):
        return (y * self.std) + self.mean

In [ ]:
for channel in dataset:
    channel.remove_randomly(pct=0.3)

# drop relative ranges to simulate sensor failure
dataset[0].remove_rel_range(0.2, 0.3)
dataset[1].remove_rel_range(0.8, None)
dataset[2].remove_rel_range(0.9, None)
dataset[3].remove_rel_range(0.8, None)
dataset[4].remove_rel_range(None, 0.2)


for channel in dataset:
    channel.transform(mogptk.TransformDetrend(degree=1))
    channel.transform(Transform_Whiten())
dataset.plot();

Then we create the test inputs and output using the atribute `mogptk.Data.mask` which contain booleans indicating if the point mas removed or not, removed points will be treated as test points.

In [ ]:
x_test = [channel.X[~channel.mask] for channel in dataset]

y_test = [channel.Y[~channel.mask] for channel in dataset]

# Models

We will use one of each of the models in the toolkit, starting with independent Gaussian processes with [SM](https://games-uchile.github.io/MultiOutputGP-Toolkit/sm.html) kernel.

### 1-Independent GP with spectral mixture kernel

For each channel we will use Q=5

In [ ]:
Q = 5 # number of mixtures
n_channels = len(dataset)

# list of the models
igp_list = []

for i in range(n_channels):
    model = mogptk.SM(dataset[i], Q=Q)
    model.estimate_params('BNSE')
    
    model.train(method='L-BFGS-B', maxiter=3000, tol=1e-30)
    igp_list.append(model)

using the function [`mogptk.test_errors`](https://games-uchile.github.io/MultiOutputGP-Toolkit/errors.html#mogptk.errors.test_errors) given a test input and output it calculates:

* Mean Absolute Error (MAE)
* mean-Normalized Mean Absolute Error (nMAE)
* Root Mean Squared Error (RMSE)
* mean-Normalized Mean Absolute Squared Error (nRMSE)


If only the raw values $(y_{pred} - y_{true})$ are wanted, the flag `raw_errors` must be set to `True`.

Multiple models for the same test set (x_test, y_test) can be passed at once, the result will be a list with one element for each model passed, where each element is another list of length equal to the number or channels where each element is the error for said model, said channel.

This enables to obtain errors for multiple models for the same test set, where each channel can have different number of test points.

In [ ]:
for model in igp_list:
    model.predict(dataset[0].X)
    model.plot_prediction(figsize=(10, 2))

In [ ]:
igp_errors = np.zeros((n_channels, 4))

for i in range(n_channels):
    igp_errors[i, :] = mogptk.test_errors(igp_list[i], x_test=x_test[i], y_test=y_test[i])[0][0]
    
pd.DataFrame(igp_errors,
             columns=['MAE', 'nMAE', 'RMSE', 'nRMSE'],
             index=cols)

## 2-Multioutput Spectral Mixture (MOSM)

Next we use the multioutput spectral mixture (Parra et al, 2016) to model the measurements

In [ ]:
x_pred = [channel.X for channel in dataset]

In [ ]:
model_mosm = mogptk.MOSM(dataset, Q=4)
model_mosm.estimate_params('BNSE')

model_mosm.train(method='L-BFGS-B', maxiter=3000, tol=1e-15)

model_mosm.predict(x_pred)
model_mosm.plot_prediction(grid=(5, 1), figsize=(10, 10), title='MOSM on Air Quality Data', names=cols)

## 3-Cross Spectral Mixture (CSM)

Then we use the cross spectral mixture kernel (Ulrich et al, 2015)

In [ ]:
model_csm = mogptk.CSM(data, Q=4)
model_csm.estimate_params()

model_csm.train(method='L-BFGS-B', maxiter=3000, tol=1e-15)

model_csm.predict(x_pred)
model_csm.plot_prediction(grid=(5, 1), figsize=(10, 10), title='CSM on Air Quality Data', names=cols)

## 4-Spectral Mixture - Linear Model of Corregionalization (SM-LMC)

Lastly we fit the Spectral mixture- linear model of corregionalization (Wilson, 2014)

In [ ]:
model_smlmc = mogptk.SM_LMC(data, Q=4)
model_smlmc.estimate_params()

model_smlmc.train(method='L-BFGS-B', maxiter=3000, tol=1e-15)

model_smlmc.predict(x_pred)
model_smlmc.plot_prediction(grid=(5, 1), figsize=(10, 10), title='CSM on Air Quality Data', names=cols)

## 5-Convolutional Gaussian (CONV)

In [ ]:
model_conv = mogptk.CG(data, Q=4)
model_conv.estimate_params()

model_conv.train(method='L-BFGS-B', maxiter=3000, tol=1e-15)

model_cov.predict(x_pred)
model_conv.plot_prediction(grid=(5, 1), figsize=(10, 10), title='CONV on Air Quality Data', names=cols)

# Compare errors

In [ ]:
errors = mogptk.test_errors(model_smlmc, model_conv, model_csm, model_mosm,
                           x_test=x_test, y_test=y_test)

We will take the mean MAE, nMAE, RMSE, nRMSE for all the channels and compare all the models

In [ ]:
pd.DataFrame(np.c_[igp_errors.mean(0),
                   np.array(errors[0]).mean(0),
                   np.array(errors[1]).mean(0),
                   np.array(errors[2]).mean(0),
                   np.array(errors[3]).mean(0),
                  ],
             columns=['mean MAE', 'mean nMAE', 'mean RMSE', 'mean nRMSE'],
             index=['IGP', 'SM-LMC', 'CONV', 'CSM', 'MOSM'])